In [1]:
import os
import sys
import numpy as np
import pandas as pd
import csv as csv
import tqdm.auto as tqdm
from multiprocessing import Pool
from importlib import reload  
import support as sup
reload(sup)
sys.path.insert(0, '../icenumerics')

import icenumerics as ice
from icenumerics.geometry import ordering 
import magcolloids as mgc

import matplotlib.pyplot as plt
import time
ureg = ice.ureg

In [2]:
output = "/home/carolina/CairoLattice_ACI_PROJECT/Scripts/Output_Script-8.1"

# Cairo Lattice 

    In this script, I will run simulations with periodic boundaries. I will run simulations in parallel.

In [3]:
def do_everything(exp_entry):

    ureg = ice.ureg

    e = exp_entry[1].e 
    l = exp_entry[1].l
    a = 19.5458*ureg.um

    # <To change the seed of the thermal noise>

    np.random.seed()

    # <Introduce the parameters for the simulation>

    sp = sup.spins()
    ureg = ice.ureg
    sp.create_lattice(geometry = "cairo",size =[l,l],lattice_constant=a , border="periodic")
    sp.order_spins(ordering.random_ordering)

    particle = ice.particle(radius = 5*ureg.um,
                 susceptibility = 0.005,
                 diffusion = 0.125*ureg.um**2/ureg.s,
                 temperature = 300*ureg.K,
                 density = 1000*ureg.kg/ureg.m**3)

    trap_long = ice.trap(trap_sep = 10*ureg.um,
                   height = 25*ureg.pN*ureg.nm,
                   stiffness = 0.1*ureg.fN/ureg.nm)

    trap_short = ice.trap(trap_sep = 4.5258*ureg.um,
                   height = 25*ureg.pN*ureg.nm,
                   stiffness = 0.1*ureg.fN/ureg.nm)

    traps = []

    # <In this loop, I will define which traps have to be shortere>

    for s in sp:

        if ( s.direction[0].magnitude == a.magnitude or s.direction[0].magnitude == -a.magnitude or 
            s.direction[1].magnitude == a.magnitude or s.direction[1].magnitude == -a.magnitude ):

            traps.append(trap_short)

        else:

            traps.append(trap_long) 

    col = ice.colloidal_ice(sp, particle, traps, height_spread = 0, susceptibility_spread = 0.1)

    col.pad_region(4.2313*ureg.um) # Why is this important?? Whitout this line the simulation do not run.
                              # It adds a space in the borders. With periodic bounds it needs to be adjusted.
                              # It depends on the lattice parameters. Here = (473/890)*l-trap_sep_long


    # <Introduce the simulation parameters>

    world = ice.world(
        field = 25*ureg.mT,
        temperature = 300*ureg.K,
        dipole_cutoff = 200*ureg.um,
        boundaries = ["p", "p", "p"])

    change_m_time = 2000*ureg.s
    total_time = 2500*ureg.s
    col.simulate(world,
                 name = "ACI_CairoLattice_25mT-2500s_PeriodicBound_l%u_exp%u"%(l,e),
                 include_timestamp = False,
                 targetdir = output,
                 framerate = 1*ureg.Hz,
                 timestep = 10*ureg.ms,
                 run_time = total_time,
                 output = ["x","y","z","mux","muy","muz"])


    col.sim.field.fieldz = "(time<2000000000)*v_Bmag*time/%f+(time>=2000000000)*v_Bmag"%(change_m_time.to(ureg.us).magnitude)
    col.run_simulation() 
    col.load_simulation(slice(0,None,1))

    #f, (ax1) = plt.subplots(1,1,figsize = (15,15))
    #col.display(ax1)

    name = os.path.split(col.sim.base_name)[1]
    with open(os.path.join(output,"index_ACI_CairoLattice_25mT-2500s_PeriodicBound.dat"),'a',newline='') as file:
        writer = csv.writer(file,delimiter='\t')
        writer.writerow([name, l, e])

In [4]:
e =  np.arange(0,10)
l =  [10]
L, E = np.meshgrid(l,e)
experiments = pd.DataFrame({"e":E.flatten(),"l":L.flatten()})
experiments

,e,l
0,0,10
1,1,10
2,2,10
3,3,10
4,4,10
5,5,10
6,6,10
7,7,10
8,8,10
9,9,10


In [5]:
# %%time
if __name__ ==  '__main__': 
    num_processors = 10
    p=Pool(processes = num_processors)
    
    ## Create index text file
    if not os.path.exists(output):
        os.makedirs(output)
    with open(os.path.join(output,"index_ACI_CairoLattice_25mT-2500s_PeriodicBound.dat"),'w',newline='') as file:
        writer = csv.writer(file,delimiter='\t')
        writer.writerow(["filename", "l", "exp"])
        
    list(tqdm.tqdm(p.imap(do_everything, experiments.iterrows()), total=len(experiments)))

/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:283: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_centers[:,1] = centersY


/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:283: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_centers[:,1] = centersY
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:283: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_centers[:,1] = centersY
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:283: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_centers[:,1] = centersY
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:283: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_centers[:,1] = centersY
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:283: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_centers[:,1] = centersY
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:283: UnitStrippedWarning: The unit o

scanning file


scanning file


scanning file


...

scanning file


scanning file


scanning file


scanning file


scanning file
